In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [2]:
torch.__version__

'2.1.0+cu118'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!cd drive/MyDrive/MNIST


In [5]:
batch_size = 32

In [6]:
train_df = pd.read_csv('drive/MyDrive/MNIST/train.csv')
X_val = torch.tensor(train_df.iloc[:1000, 1:].values, dtype=torch.float32)
X_train = torch.tensor(train_df.iloc[1000:, 1:].values, dtype=torch.float32)
y_val = torch.tensor(train_df.loc[:999, 'label'].values)
y_train = torch.tensor(train_df.loc[1000:, 'label'].values)

X_train /= 255.0
X_val /= 255.0


test_df = pd.read_csv('drive/MyDrive/MNIST/test.csv')
X_test = torch.tensor(test_df.values, dtype=torch.float32)
X_test /= 255.0


In [7]:
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [10]:
input_size = X_train.shape[1]
hidden_size1 = 128
hidden_size2 = 64
num_classes = 10

model = NeuralNetwork(input_size, hidden_size1, hidden_size2, num_classes)


In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.2)

In [12]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.inference_mode():
        val_predictions = []
        for batch_x, batch_y in val_loader:
            outputs = model(batch_x)
            _, predicted = torch.max(outputs, 1)
            val_predictions.extend(predicted.numpy())
        accuracy = accuracy_score(y_val, val_predictions)
        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {accuracy * 100:.2f}%')

Epoch [1/10], Validation Accuracy: 93.90%
Epoch [2/10], Validation Accuracy: 89.40%
Epoch [3/10], Validation Accuracy: 96.40%
Epoch [4/10], Validation Accuracy: 97.40%
Epoch [5/10], Validation Accuracy: 97.30%
Epoch [6/10], Validation Accuracy: 97.30%
Epoch [7/10], Validation Accuracy: 93.80%
Epoch [8/10], Validation Accuracy: 97.30%
Epoch [9/10], Validation Accuracy: 97.60%
Epoch [10/10], Validation Accuracy: 98.10%


In [13]:
with torch.inference_mode():
  output = model(X_test)
  _, predictions = torch.max(output, axis = 1)

print(predictions)

tensor([2, 0, 9,  ..., 3, 9, 2])


In [14]:
ImageId = np.arange(predictions.shape[0]) + 1
submission = pd.DataFrame({'ImageId': ImageId, 'Label': predictions})
submission.to_csv('submission.csv', index=False)